In [1]:
import numpy as np
import scipy as sp
from scipy import random

In [2]:
def sum_elem_product_noncvx(A,B):
    return np.sum(np.multiply(A,B))

sensors= np.matrix([
    [0.1, 0.3],
    [0, 0.4]
])

anchors = np.matrix([
    [ 1, 0],
    [-1, 0],
    [ 0, 2]
])

A_1 = np.matrix([[1], [0], [0], [0]]) * \
        np.transpose(np.matrix([[1], [0], [0], [0]]))
A_2 = np.matrix([[0], [1], [0], [0]]) * \
        np.transpose(np.matrix([[0], [1], [0], [0]]))
A_3 = np.matrix([[1], [1], [0], [0]]) * \
        np.transpose(np.matrix([[1], [1], [0], [0]]))

A_4 = np.vstack((np.transpose(anchors[0]), -1, 0)) * \
        np.transpose(np.vstack((np.transpose(anchors[0]), -1, 0)))
A_5 = np.vstack((np.transpose(anchors[1]), -1, 0)) * \
        np.transpose(np.vstack((np.transpose(anchors[1]), -1, 0)))

A_6 = np.vstack((np.transpose(anchors[1]), 0, -1)) * \
        np.transpose(np.vstack((np.transpose(anchors[1]), 0, -1)))
A_7 = np.vstack((np.transpose(anchors[2]), 0, -1)) * \
        np.transpose(np.vstack((np.transpose(anchors[2]), 0, -1)))

A_8 = np.matrix([[0], [0], [1], [-1]]) * \
        np.transpose(np.matrix([[0], [0], [1], [-1]]))


A = [A_1, A_2, A_3, A_4, A_5, A_6, A_7, A_8]

b = np.matrix([
    [1],
    [1],
    [2],
    
    [np.linalg.norm(anchors[0] - sensors[0])**2],
    [np.linalg.norm(anchors[1] - sensors[0])**2],
    
    [np.linalg.norm(anchors[1] - sensors[1])**2],
    [np.linalg.norm(anchors[2] - sensors[1])**2],
    
    [np.linalg.norm(sensors[0] - sensors[1])**2]
])

def make_AX(X):
    output = []
    for i, A_i in enumerate(A):
        output.append(sum_elem_product_noncvx(A_i, X))
    return np.transpose(np.matrix(output))

def At_y(y):
    output = np.zeros((4,4))
    for i, A_i in enumerate(A):
        output += np.multiply(A_i, y[i])
    return output

def grad(X):
    return At_y(make_AX(X) - b)

#prepare iterations
dim = np.shape(sensors)[0]
num_sensors = np.shape(sensors)[1]
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
X_k = X_0
Beta = 100
k = 0
check = 1000
max_iter = 20000

while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    [eigenvalues, V] = np.linalg.eig(X_k1)
    eigenvalues[eigenvalues < 0] = 0
    Lambda = np.diag(eigenvalues)
    X_k1 = np.dot(np.dot(V, Lambda), np.transpose(V))
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1

    

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [119]:
from scipy.linalg import ldl

def ldl_decomp(A):
    A = np.matrix(A)

    S = np.diag(np.diag(A))
    Sinv = np.diag(1/np.diag(A))
    D = np.matrix(S.dot(S))
    Lch = np.linalg.cholesky(A)
    L = np.matrix(Lch.dot(Sinv))
    return L, D

#prepare iterations
dim = np.shape(sensors)[0]
num_sensors = np.shape(sensors)[1]
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
X_k = X_0
Beta = 10000000
k = 0
check = 1000
max_iter = 20000

    
while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    [L, D, _] = sp.linalg.ldl(X_k1)
    D[D < 0] = 0
    D_psd = np.diag(D)
    X_k1 = np.dot(np.dot(L, D), np.transpose(L))
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1

ImportError: cannot import name 'ldl'

In [135]:
import scipy
from scipy.linalg import eigh
def pca(X):
    [evalues, evectors] = scipy.linalg.eigh(X)
    
    idx = evalues.argsort()[::-1]   
    evalues = evalues[idx]
    evectors = evectors[:,idx]
    
    matrix = np.zeros(X.shape)
    width = evalues.shape[0]
    for i in range(3):
        index = width - i - 1
        if index >= 0 and evalues[index] > 0:
            matrix += evalues[index] * np.outer(evectors[:, index], evectors[:, index])
    return matrix

In [136]:
dim = np.shape(sensors)[0]
num_sensors = np.shape(sensors)[1]
thing = random.rand(dim+num_sensors,dim+num_sensors)
X_0 = np.dot(thing,thing.transpose())
X_k = X_0
Beta = 100
k = 0
check = 1000
max_iter = 20000

    
while check > 10**-8 and k < max_iter:
    
    #get next iterate from descent
    X_k1 = X_k - (1/Beta) * grad(X_k)
    
    #project back into cone with eigen decomp
    X_k1 = pca(X_k1) 
    
    #set up for next iteration
    check = np.linalg.norm(X_k1 - X_k)  
    X_k = X_k1
    
    if(k%1000 == 0):
        print(k)
    k = k+1

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000


In [130]:
LA

<module 'numpy.linalg' from '/Users/lsmoore/anaconda3/lib/python3.6/site-packages/numpy/linalg/__init__.py'>

In [134]:
print("Real sensors: ")
print(sensors)
print("Generated Sensors: ")
generated = np.transpose(X_k[0:dim, dim:dim+num_sensors])
print(generated)


Real sensors: 
[[ 0.1  0.3]
 [ 0.   0.4]]
Generated Sensors: 
[[ 0.0154571  -0.00882466]
 [ 0.00524817  0.00990563]]
